# Mosaico 1996

Script trabajado por Francisco Albornoz (2021-04-13) / Última modificación / Mosaicos Patagonia

2021-08-05 13:07:09 

In [1]:
import ee
import geemap
import geemap.chart as chart
import os
import folium

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

In [2]:
Map = geemap.Map(center = [-45, -70], zoom = 4)
Map2 = geemap.Map(center = [-45, -70], zoom = 4)
Map3 = geemap.Map(center = [-45, -70], zoom = 4)
Map4 = geemap.Map(center = [-45, -70], zoom = 4)
Map5 = geemap.Map(center = [-45, -70], zoom = 4)
Map6 = geemap.Map(center = [-45, -70], zoom = 4)
Map7 = geemap.Map(center = [-45, -70], zoom = 4)
Map8 = geemap.Map(center = [-45, -70], zoom = 4)
Map9 = geemap.Map(center = [-45, -70], zoom = 4)
Map10 = geemap.Map(center = [-45, -70], zoom = 4)

## Área de Estudio

In [3]:
work_dir = os.path.expanduser('../../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_nueva.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)

## Colors Bars

In [4]:
import geemap.colormaps as cm

In [5]:
# cm.palettes.terrain

In [6]:
color_ndvi = cm.palettes.ndvi

In [7]:
color_ndwi = cm.palettes.ndwi

In [8]:
color_terrain = cm.palettes.terrain

In [9]:
color_glaciar = cm.palettes.PuBuGn_r

In [10]:
color_nubes = cm.palettes.inferno

In [11]:
# cm.list_colormaps()

In [12]:
# cm.plot_colormaps(width=12, height=0.4)

## Funciones

In [13]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [14]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('B11')
    return image.addBands(ndvi)

In [15]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B2','B4']).rename('NDWI')
    return image.addBands(ndwi)

In [16]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B2','B3']).rename('NDGI')
    return image.addBands(ndgi)

In [17]:
def addSAVI(image):
    savi = image.expression(
        '((b4 - b3)/(b4 + b3 + 0.5))*1.5', {
            'b3':image.select('B3'),
            'b4':image.select('B4')
        }) 
    savi= savi.rename('SAVI')
    return image.addBands(savi)

In [18]:
def addNDSI(image):
    ndsi = image.normalizedDifference(['B2','B5']).rename('NDSI')
    return image.addBands(ndsi)

In [19]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMM'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [20]:
def Cloud40(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(40)) 

In [21]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [22]:
def Cloud20(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(20)) 

In [23]:
def Cloud10(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(10))

In [24]:
def addCloud(image):
    nube = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.addBands(ee.Image(nube.select(['cloud'])).rename('nube'))

## Colecciones

### 1996

In [25]:
start_date = '1995-10-01'
end_date = '1996-03-31'

l5_1996 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [26]:
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [27]:
vis_nieve = {
  'bands': ['B5', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [28]:
coleccion1996 = l5_1996.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [29]:
QM_NDVI_1996 = coleccion1996.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_1996 = coleccion1996.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_1996 = coleccion1996.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_1996 = coleccion1996.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_1996 = coleccion1996.qualityMosaic('NDSI') # Normalized difference snow index

In [30]:
Map.addLayer(QM_NDVI_1996, vizParams, 'QM de NDVI 1996')
Map.addLayer(QM_NDWI_1996, vizParams, 'QM de NDWI 1996')
Map.addLayer(QM_SAVI_1996, vizParams, 'QM de SAVI 1996')
Map.addLayer(QM_NDGI_1996, vis_nieve, 'QM de NDGI 1996')
Map.addLayer(QM_NDSI_1996, vis_nieve, 'QM de NDSI 1996')
Map 

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [31]:
c1996_sn = coleccion1996.map(Cloud20)
NDVI_1996_sn = c1996_sn.qualityMosaic('B11')
NDWI_1996_sn = c1996_sn.qualityMosaic('NDWI')
SAVI_1996_sn = c1996_sn.qualityMosaic('SAVI')
NDGI_1996_sn = c1996_sn.qualityMosaic('NDGI')
NDSI_1996_sn = c1996_sn.qualityMosaic('NDSI')

In [32]:
Map2.addLayer(NDVI_1996_sn, vizParams, 'QMsn de NDVI 1996')
Map2.addLayer(NDWI_1996_sn, vizParams, 'QMsn de NDWI 1996')
Map2.addLayer(SAVI_1996_sn, vizParams, 'QMsn de SAVI 1996')
Map2.addLayer(NDGI_1996_sn, vis_nieve, 'QMsn de NDGI 1996')
Map2.addLayer(NDSI_1996_sn, vis_nieve, 'QMsn de NDSI 1996')
Map2

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 1997

In [33]:
start_date = '1996-10-01'
end_date = '1997-03-31'

l5_1997 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [34]:
coleccion1997 = l5_1997.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [35]:
QM_NDVI_1997 = coleccion1997.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_1997 = coleccion1997.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_1997 = coleccion1997.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_1997 = coleccion1997.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_1997 = coleccion1997.qualityMosaic('NDSI') # Normalized difference snow index

In [36]:
Map.addLayer(QM_NDVI_1997, vizParams, 'QM de NDVI 1997')
Map.addLayer(QM_NDWI_1997, vizParams, 'QM de NDWI 1997')
Map.addLayer(QM_SAVI_1997, vizParams, 'QM de SAVI 1997')
Map.addLayer(QM_NDGI_1997, vis_nieve, 'QM de NDGI 1997')
Map.addLayer(QM_NDSI_1997, vis_nieve, 'QM de NDSI 1997')
Map 

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [37]:
c1997_sn = coleccion1997.map(Cloud20)
NDVI_1997_sn = c1997_sn.qualityMosaic('B11')
NDWI_1997_sn = c1997_sn.qualityMosaic('NDWI')
SAVI_1997_sn = c1997_sn.qualityMosaic('SAVI')
NDGI_1997_sn = c1997_sn.qualityMosaic('NDGI')
NDSI_1997_sn = c1997_sn.qualityMosaic('NDSI')

In [38]:
Map2.addLayer(NDVI_1997_sn, vizParams, 'QMsn de NDVI 1997')
Map2.addLayer(NDWI_1997_sn, vizParams, 'QMsn de NDWI 1997')
Map2.addLayer(SAVI_1997_sn, vizParams, 'QMsn de SAVI 1997')
Map2.addLayer(NDGI_1997_sn, vis_nieve, 'QMsn de NDGI 1997')
Map2.addLayer(NDSI_1997_sn, vis_nieve, 'QMsn de NDSI 1997')
Map2

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 1998

In [39]:
start_date = '1997-10-01'
end_date = '1998-03-31'

l5_1998 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [40]:
coleccion1998 = l5_1998.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [41]:
QM_NDVI_1998 = coleccion1998.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_1998 = coleccion1998.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_1998 = coleccion1998.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_1998 = coleccion1998.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_1998 = coleccion1998.qualityMosaic('NDSI') # Normalized difference snow index

In [42]:
Map3.addLayer(QM_NDVI_1998, vizParams, 'QM de NDVI 1998')
Map3.addLayer(QM_NDWI_1998, vizParams, 'QM de NDWI 1998')
Map3.addLayer(QM_SAVI_1998, vizParams, 'QM de SAVI 1998')
Map3.addLayer(QM_NDGI_1998, vis_nieve, 'QM de NDGI 1998')
Map3.addLayer(QM_NDSI_1998, vis_nieve, 'QM de NDSI 1998')
# Map3

In [43]:
c1998_sn = coleccion1998.map(Cloud20)
NDVI_1998_sn = c1998_sn.qualityMosaic('B11')
NDWI_1998_sn = c1998_sn.qualityMosaic('NDWI')
SAVI_1998_sn = c1998_sn.qualityMosaic('SAVI')
NDGI_1998_sn = c1998_sn.qualityMosaic('NDGI')
NDSI_1998_sn = c1998_sn.qualityMosaic('NDSI')

In [44]:
Map4.addLayer(NDVI_1998_sn, vizParams, 'QMsn de NDVI 1998')
Map4.addLayer(NDWI_1998_sn, vizParams, 'QMsn de NDWI 1998')
Map4.addLayer(SAVI_1998_sn, vizParams, 'QMsn de SAVI 1998')
Map4.addLayer(NDGI_1998_sn, vis_nieve, 'QMsn de NDGI 1998')
Map4.addLayer(NDSI_1998_sn, vis_nieve, 'QMsn de NDSI 1998')
Map4

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 1999 

In [45]:
start_date = '1998-10-01'
end_date = '1999-03-31'

l5_1999 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [46]:
coleccion1999 = l5_1999.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [47]:
QM_NDVI_1999 = coleccion1999.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_1999 = coleccion1999.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_1999 = coleccion1999.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_1999 = coleccion1999.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_1999 = coleccion1999.qualityMosaic('NDSI') # Normalized difference snow index

In [48]:
Map3.addLayer(QM_NDVI_1999, vizParams, 'QM de NDVI 1999')
Map3.addLayer(QM_NDWI_1999, vizParams, 'QM de NDWI 1999')
Map3.addLayer(QM_SAVI_1999, vizParams, 'QM de SAVI 1999')
Map3.addLayer(QM_NDGI_1999, vis_nieve, 'QM de NDGI 1999')
Map3.addLayer(QM_NDSI_1999, vis_nieve, 'QM de NDSI 1999')
# Map3

In [49]:
c1999_sn = coleccion1999.map(Cloud20)
NDVI_1999_sn = c1999_sn.qualityMosaic('B11')
NDWI_1999_sn = c1999_sn.qualityMosaic('NDWI')
SAVI_1999_sn = c1999_sn.qualityMosaic('SAVI')
NDGI_1999_sn = c1999_sn.qualityMosaic('NDGI')
NDSI_1999_sn = c1999_sn.qualityMosaic('NDSI')

In [50]:
Map4.addLayer(NDVI_1999_sn, vizParams, 'QMsn de NDVI 1999')
Map4.addLayer(NDWI_1999_sn, vizParams, 'QMsn de NDWI 1999')
Map4.addLayer(SAVI_1999_sn, vizParams, 'QMsn de SAVI 1999')
Map4.addLayer(NDGI_1999_sn, vis_nieve, 'QMsn de NDGI 1999')
Map4.addLayer(NDSI_1999_sn, vis_nieve, 'QMsn de NDSI 1999')
Map4

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 2000

In [51]:
start_date = '1999-10-01'
end_date = '2000-03-31'

l5_2000 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [52]:
coleccion2000 = l5_2000.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [53]:
QM_NDVI_2000 = coleccion2000.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_2000 = coleccion2000.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_2000 = coleccion2000.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_2000 = coleccion2000.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_2000 = coleccion2000.qualityMosaic('NDSI') # Normalized difference snow index

In [54]:
Map3.addLayer(QM_NDVI_2000, vizParams, 'QM de NDVI 2000')
Map3.addLayer(QM_NDWI_2000, vizParams, 'QM de NDWI 2000')
Map3.addLayer(QM_SAVI_2000, vizParams, 'QM de SAVI 2000')
Map3.addLayer(QM_NDGI_2000, vis_nieve, 'QM de NDGI 2000')
Map3.addLayer(QM_NDSI_2000, vis_nieve, 'QM de NDSI 2000')
# Map3

In [55]:
c2000_sn = coleccion2000.map(Cloud20)
NDVI_2000_sn = c2000_sn.qualityMosaic('B11')
NDWI_2000_sn = c2000_sn.qualityMosaic('NDWI')
SAVI_2000_sn = c2000_sn.qualityMosaic('SAVI')
NDGI_2000_sn = c2000_sn.qualityMosaic('NDGI')
NDSI_2000_sn = c2000_sn.qualityMosaic('NDSI')

In [56]:
Map4.addLayer(NDVI_2000_sn, vizParams, 'QMsn de NDVI 2000')
Map4.addLayer(NDWI_2000_sn, vizParams, 'QMsn de NDWI 2000')
Map4.addLayer(SAVI_2000_sn, vizParams, 'QMsn de SAVI 2000')
Map4.addLayer(NDGI_2000_sn, vis_nieve, 'QMsn de NDGI 2000')
Map4.addLayer(NDSI_2000_sn, vis_nieve, 'QMsn de NDSI 2000')
Map4

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

#### Colección agua 1996-1999

In [57]:
start_date = '1995-10-01'
end_date = '2000-03-31'

l5_sur = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [58]:
coleccionsur = l5_sur.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

In [59]:
QM_NDVI_sur = coleccionsur.qualityMosaic('B11') # Normalized difference vegetation Index
QM_NDWI_sur = coleccionsur.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_sur = coleccionsur.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_sur = coleccionsur.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_sur = coleccionsur.qualityMosaic('NDSI') # Normalized difference snow index

In [60]:
Map3.addLayer(QM_NDVI_sur, vizParams, 'QM de NDVI sur')
Map3.addLayer(QM_NDWI_sur, vizParams, 'QM de NDWI sur')
Map3.addLayer(QM_SAVI_sur, vizParams, 'QM de SAVI sur')
Map3.addLayer(QM_NDGI_sur, vis_nieve, 'QM de NDGI sur')
Map3.addLayer(QM_NDSI_sur, vis_nieve, 'QM de NDSI sur')
# Map3

In [61]:
csur_sn = coleccionsur.map(Cloud20)
NDVI_sur_sn = csur_sn.qualityMosaic('B11')
NDWI_sur_sn = csur_sn.qualityMosaic('NDWI')
SAVI_sur_sn = csur_sn.qualityMosaic('SAVI')
NDGI_sur_sn = csur_sn.qualityMosaic('NDGI')
NDSI_sur_sn = csur_sn.qualityMosaic('NDSI')

In [62]:
Map4.addLayer(NDVI_sur_sn, vizParams, 'QMsn de NDVI sur')
Map4.addLayer(NDWI_sur_sn, vizParams, 'QMsn de NDWI sur')
Map4.addLayer(SAVI_sur_sn, vizParams, 'QMsn de SAVI sur')
Map4.addLayer(NDGI_sur_sn, vis_nieve, 'QMsn de NDGI sur')
Map4.addLayer(NDSI_sur_sn, vis_nieve, 'QMsn de NDSI sur')
Map4

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 1996 - 2000

In [63]:
coleccion1996_2000 = ee.ImageCollection(l5_2000.merge(l5_1999).merge(l5_1998).merge(l5_1997).merge(l5_1996))

In [64]:
l5_summer = coleccion1996_2000.map(addNDVI).map(addNDWI).map(addNDGI).map(addSAVI).map(addNDSI).map(addDate).map(addCloud)

### Tier 1

In [65]:
Tier1_sn = l5_summer.map(Cloud10)

In [66]:
QM_NDVI_T1 = Tier1_sn.qualityMosaic('B11')# Normalized difference vegetation Index
QM_NDWI_T1 = Tier1_sn.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_T1 = Tier1_sn.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_T1 = Tier1_sn.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_T1 = Tier1_sn.qualityMosaic('NDSI') # Normalized difference snow index

### Tier 2

In [67]:
Tier2_sn = l5_summer.map(Cloud20)

In [68]:
QM_NDVI_T2 = Tier2_sn.qualityMosaic('B11')# Normalized difference vegetation Index
QM_NDWI_T2 = Tier2_sn.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_T2 = Tier2_sn.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_T2 = Tier2_sn.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_T2 = Tier2_sn.qualityMosaic('NDSI') # Normalized difference snow index

### Tier 3

In [69]:
Tier3_sn = l5_summer.map(Cloud30)

In [70]:
QM_NDVI_T3 = Tier3_sn.qualityMosaic('B11')# Normalized difference vegetation Index
QM_NDWI_T3 = Tier3_sn.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_T3 = Tier3_sn.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_T3 = Tier3_sn.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_T3 = Tier3_sn.qualityMosaic('NDSI') # Normalized difference snow index

### Tier 4

In [71]:
Tier4_sn = l5_summer.map(Cloud40)

In [72]:
QM_NDVI_T4 = Tier4_sn.qualityMosaic('B11')# Normalized difference vegetation Index
QM_NDWI_T4 = Tier4_sn.qualityMosaic('NDWI') # Normalize difference water index
QM_SAVI_T4 = Tier4_sn.qualityMosaic('SAVI') # Soil Adjusted Vegetation Index 
QM_NDGI_T4 = Tier4_sn.qualityMosaic('NDGI') # Normalized difference glacier index
QM_NDSI_T4 = Tier4_sn.qualityMosaic('NDSI') # Normalized difference snow index

## Indices por colección

### Índices 1996

In [73]:
NDVI_1996 = NDVI_1996_sn.select('B11')
NDWI_1996 = NDWI_1996_sn.select('NDWI') 
SAVI_1996 = SAVI_1996_sn.select('SAVI')
NDGI_1996 = NDGI_1996_sn.select('NDGI')
NDSI_1996 = NDSI_1996_sn.select('NDSI')

In [74]:
Map7.addLayer(NDVI_1996, {'palette': color_terrain}, 'QMsn de NDVI 1996')
Map7.addLayer(NDWI_1996, {'palette': color_ndwi}, 'QMsn de NDWI 1996')
Map7.addLayer(SAVI_1996, {'palette': color_ndvi}, 'QMsn de SAVI 1996')
# Map7.addLayer(NDGI_1996, {'palette': color_glaciar}, 'QMsn de NDGI 1996')
# Map7.addLayer(NDSI_1996, {'palette': color_ndwi}, 'QMsn de NDSI 1996')
Map7

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Índices 1997

In [75]:
NDVI_1997 = NDVI_1997_sn.select('B11')
NDWI_1997 = NDWI_1997_sn.select('NDWI') 
SAVI_1997 = SAVI_1997_sn.select('SAVI')
NDGI_1997 = NDGI_1997_sn.select('NDGI')
NDSI_1997 = NDSI_1997_sn.select('NDSI')

In [76]:
Map7.addLayer(NDVI_1997, {'palette': color_terrain}, 'QMsn de NDVI 1997')
Map7.addLayer(NDWI_1997, {'palette': color_ndwi}, 'QMsn de NDWI 1997')
Map7.addLayer(SAVI_1997, {'palette': color_ndvi}, 'QMsn de SAVI 1997')
# Map7.addLayer(NDGI_1997, {'palette': color_glaciar}, 'QMsn de NDGI 1997')
# Map7.addLayer(NDSI_1997, {'palette': color_ndwi}, 'QMsn de NDSI 1997')
Map7

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Índices 1998

In [77]:
NDVI_1998 = NDVI_1998_sn.select('B11')
NDWI_1998 = NDWI_1998_sn.select('NDWI') 
SAVI_1998 = SAVI_1998_sn.select('SAVI')
NDGI_1998 = NDGI_1998_sn.select('NDGI')
NDSI_1998 = NDSI_1998_sn.select('NDSI')

In [78]:
Map8.addLayer(NDVI_1998, {'palette': color_terrain}, 'QMsn de NDVI 1998')
Map8.addLayer(NDWI_1998, {'palette': color_ndwi}, 'QMsn de NDWI 1998')
Map8.addLayer(SAVI_1998, {'palette': color_ndvi}, 'QMsn de SAVI 1998')
# Map8.addLayer(NDGI_1998, {'palette': color_glaciar}, 'QMsn de NDGI 1998')
# Map8.addLayer(NDSI_1998, {'palette': color_ndwi}, 'QMsn de NDSI 1998')
Map8

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Índices 1999

In [79]:
NDVI_1999 = NDVI_1999_sn.select('B11')
NDWI_1999 = NDWI_1999_sn.select('NDWI') 
SAVI_1999 = SAVI_1999_sn.select('SAVI')
NDGI_1999 = NDGI_1999_sn.select('NDGI')
NDSI_1999 = NDSI_1999_sn.select('NDSI')

In [80]:
Map8.addLayer(NDVI_1999, {'palette': color_terrain}, 'QMsn de NDVI 1999')
Map8.addLayer(NDWI_1999, {'palette': color_ndwi}, 'QMsn de NDWI 1999')
Map8.addLayer(SAVI_1999, {'palette': color_ndvi}, 'QMsn de SAVI 1999')
# Map8.addLayer(NDGI_1999, {'palette': color_glaciar}, 'QMsn de NDGI 1999')
# Map8.addLayer(NDSI_1999, {'palette': color_ndwi}, 'QMsn de NDSI 1999')
Map8

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Generación de Mosaico

In [81]:
mosaic = ee.ImageCollection([QM_NDVI_sur,QM_NDVI_1996]).mosaic()

In [82]:
mosaic_landsat = mosaic.set('SENSOR_ID', 'OLI_TIRS')

In [83]:
cloud_mosaic = ee.Algorithms.Landsat.simpleCloudScore(mosaic_landsat)

In [84]:
cloud_mosaic = cloud_mosaic.select('cloud')

In [85]:
palette_nubes = ['#1a9850', '#66bd63', '#a6d96a', '#d9ef8b', '#fee08b', '#fdae61', '#f46d43', '#d73027']

In [86]:
resultados= [cloud_mosaic, mosaic]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes mosaico',
    'mosaico inicial']

In [87]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [88]:
def llenado(image):
    return image.where(cloud_mosaic.gt(10),QM_NDVI_T1)

In [89]:
img1 = llenado(mosaic)

In [90]:
img1_landsat = img1.set('SENSOR_ID', 'OLI_TIRS')
cloud_img1 = ee.Algorithms.Landsat.simpleCloudScore(img1_landsat)
cloud_img1 = cloud_img1.select('cloud')

In [91]:
resultados= [cloud_img1, img1]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 1',
    'Mosaico 1']

In [92]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [93]:
def llenado2(image):
    return image.where(cloud_img1.gt(10), QM_NDVI_T2)

In [94]:
img2 = llenado2(img1)

In [95]:
img2_landsat = img2.set('SENSOR_ID', 'OLI_TIRS')
cloud_img2 = ee.Algorithms.Landsat.simpleCloudScore(img2_landsat)
cloud_img2 = cloud_img2.select('cloud')

In [96]:
resultados= [cloud_img2, img2]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 2',
    'Mosaico 2']

In [97]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [98]:
def llenado3(image):
    return image.where(cloud_img2.gt(10), QM_NDVI_T3)

In [99]:
img3 = llenado3(img2)

In [100]:
img3_landsat = img3.set('SENSOR_ID', 'OLI_TIRS')
cloud_img3 = ee.Algorithms.Landsat.simpleCloudScore(img3_landsat)
cloud_img3 = cloud_img3.select('cloud')

In [101]:
resultados= [cloud_img3, img3]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 3',
    'Mosaico 3']

In [102]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [103]:
def llenado4(image):
    return image.where(cloud_img3.gt(10), QM_NDVI_T4)

In [104]:
img4 = llenado4(img3)

In [105]:
img4_landsat = img4.set('SENSOR_ID', 'OLI_TIRS')
cloud_img4 = ee.Algorithms.Landsat.simpleCloudScore(img4_landsat)
cloud_img4 = cloud_img4.select('cloud')

In [106]:
resultados= [cloud_img4, img4]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 4',
    'Mosaico 4']

In [107]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [108]:
hansenImage = ee.Image('UMD/hansen/global_forest_change_2015')
datamask = hansenImage.select('datamask')

In [109]:
def llenado5(image):
    return image.where(datamask.eq(2), QM_NDWI_T3)

In [110]:
img5 = llenado5(img4)

In [111]:
img5_landsat = img5.set('SENSOR_ID', 'OLI_TIRS')
cloud_img5 = ee.Algorithms.Landsat.simpleCloudScore(img5_landsat)
cloud_img5 = cloud_img5.select('cloud')

In [112]:
resultados= [cloud_img5, img5]
vis_params2=  [
    {'palette':palette_nubes}, 
    vizParams]
labels = [
    'Nubes Mosaico 5',
    'Mosaico 5']

In [113]:
geemap.linked_maps(rows=1, cols=2, height="400px", center=[-45, -75], zoom=4, 
                   ee_objects=resultados, vis_params=vis_params2, labels=labels, label_position="topright")

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

In [114]:
Map10.addLayer(img4, vizParams, 'Mosaico 4')
Map10.addLayer(img5, vizParams, 'Mosaico 5')
Map10

Map(center=[-45, -70], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [115]:
img5_b = img5.select(['B1','B2','B3','B4','B5','B6','B7'])

In [116]:
molde = os.path.join(work_dir, 'molde.shp')
molde_grande = geemap.shp_to_ee(molde)

In [117]:
partes_1 = os.path.join(work_dir, '1.shp')
partes1 = geemap.shp_to_ee(partes_1)

partes_2 = os.path.join(work_dir, '2.shp')
partes2 = geemap.shp_to_ee(partes_2)

partes_3 = os.path.join(work_dir, '3.shp')
partes3 = geemap.shp_to_ee(partes_3)

partes_4 = os.path.join(work_dir, '4.shp')
partes4 = geemap.shp_to_ee(partes_4)

partes_5 = os.path.join(work_dir, '5.shp')
partes5 = geemap.shp_to_ee(partes_5)

partes_6 = os.path.join(work_dir, '6.shp')
partes6 = geemap.shp_to_ee(partes_6)

partes_7 = os.path.join(work_dir, '7.shp')
partes7 = geemap.shp_to_ee(partes_7)

partes_8 = os.path.join(work_dir, '8.shp')
partes8 = geemap.shp_to_ee(partes_8)

In [118]:
img5_b = img5_b.clip(molde_grande).unmask()

In [119]:
img5_1 = img5_b.clip(partes1).unmask()
img5_2 = img5_b.clip(partes2).unmask()
img5_3 = img5_b.clip(partes3).unmask()
img5_4 = img5_b.clip(partes4).unmask()
img5_5 = img5_b.clip(partes5).unmask()
img5_6 = img5_b.clip(partes6).unmask()
img5_7 = img5_b.clip(partes7).unmask()
img5_8 = img5_b.clip(partes8).unmask()

In [120]:
Map10.addLayer(img5_b, vizParams, 'Mosaico 5')

In [121]:
# geemap.ee_export_image_to_drive(img5_b, description='mosaico_1996', folder='export', region=patagonia_grande.geometry(), scale=30)

In [130]:
geemap.ee_export_image_to_drive(img5_1, description='mosaico_1996_1', folder='export', region=partes1.geometry(), scale=30)

Exporting mosaico_1996_1 ...


In [131]:
geemap.ee_export_image_to_drive(img5_2, description='mosaico_1996_2', folder='export', region=partes2.geometry(), scale=30)

Exporting mosaico_1996_2 ...


In [132]:
geemap.ee_export_image_to_drive(img5_3, description='mosaico_1996_3', folder='export', region=partes3.geometry(), scale=30)

Exporting mosaico_1996_3 ...


In [133]:
geemap.ee_export_image_to_drive(img5_4, description='mosaico_1996_4', folder='export', region=partes4.geometry(), scale=30)

Exporting mosaico_1996_4 ...


In [134]:
geemap.ee_export_image_to_drive(img5_5, description='mosaico_1996_5', folder='export', region=partes5.geometry(), scale=30)

Exporting mosaico_1996_5 ...


In [135]:
geemap.ee_export_image_to_drive(img5_6, description='mosaico_1996_6', folder='export', region=partes6.geometry(), scale=30)

Exporting mosaico_1996_6 ...


In [136]:
geemap.ee_export_image_to_drive(img5_7, description='mosaico_1996_7', folder='export', region=partes7.geometry(), scale=30)

Exporting mosaico_1996_7 ...


In [137]:
geemap.ee_export_image_to_drive(img5_8, description='mosaico_1996_8', folder='export', region=partes8.geometry(), scale=30)

Exporting mosaico_1996_8 ...
